# Day 4-Part 2: Grids

Grids are very important in geosciences. Basically, a grid is just a set of regularly spaced points in x and y, with some information (elevation, depth, thickness, etc.) at each point. A digital elevation model (DEM) is a good example. At each point in the DEM grid, the elevation is known. This notebook illustrates the visualization of grids in Python.

Grid data are often available, but sometimes we need to reconstruct the grid from sparse data which is not regularly spaced. This process is known as interpolation, and in this notebook we describe it as well. 

## Visualizing a seismic horizon:

We'll use a gridded seismic horizon from the Norwegian North Sea, namely the top of the Upper Permian Zechstein group, which in this area is the top of the salt. These data are in two-way travel time (TWT) in mili-seconds (ms) and were kindly provided by Dora Marin Restrepo at UiS.

First, the usual preliminaries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

Then, we load the horizon. As you can see, the data are in `npy` format, which is the standard binary file format to store numpy arrays in disk (the `numpy` `save` function save an array as a `npy`file). Loading these data is easy, and it just requires the use of the `numpy.load` function:

In [ ]:
TWT = np.load(os.path.join("..", "data", "top_zech_twt.npy"))
x = np.load(os.path.join("..", "data", "top_zech_east.npy"))
y = np.load(os.path.join("..", "data", "top_zech_north.npy"))

It is perhaps more difficult to understand, what these three arrays are. Let's start by printing their sizes:

In [ ]:
print("TWT size =", TWT.shape)
print("x size =", x.shape)
print("y size =", y.shape)

`TWT` is a large 2D array (you can tell that by the size of the `npy` file, which is 7 MB). `TWT` is the two-way travel time in ms of the horizon at each one of the points of the grid. Where the points have no data (where the horizon has not been mapped), the value of `nan` (not a number) is provided. This allows Python skipping these points.

`x` and `y` on the other hand are 1D arrays corresponding to the east (`x`) and north (`y`) UTM coordinates of the points. Since the grid is regular, there is no need to save the east and north coordinates at every point of the grid. We can reconstruct these coordinates using the `numpy.meshgrid` function as follows:

In [ ]:
X, Y = np.meshgrid(x, y)
print("X size =", X.shape)
print("Y size =", Y.shape)

Notice that this produces `X`and `Y` arrays of the same size than `TWT`. Let's explore a little bit these arrays:

In [ ]:
print("X columns =", X[0,0:5]) # print first 5 columns in first row of X
print("Y columns =", Y[0,0:5]) # print first 5 columns in first row of Y
print("X rows =", X[0:5,0]) # print first 5 rows in first column of X
print("Y rows =", Y[0:5,0]) # print first 5 rows in first column of Y

So, `X` (east) values vary with columns, while `Y` (north) values vary with rows. Every row has a constant `Y` (north) value and traverses the grid in increasing `X` (east) values, while every column has a constant `X` (east) value and traverses the grid in increasing `Y` (north) values. The figure below illustrates this:

<img src="../figures/grid.png" alt="grid" width="400"/><br><br>

Once we have the location (`X` and `Y`) and values (`TWT`) array in the format of a grid, it is easy to visualize these data. Let's draw filled contours of the top Zechstein using the `contourf` function. To make the graph more elegant, we plot the data in s instead of ms:

In [ ]:
# convert TWT to seconds
TWT /= 1000 # This is the same as TWT = TWT / 1000

# make figure
fig, ax = plt.subplots(figsize=(10,10))

# draw filled contours
plt.contourf(X, Y, TWT)

# draw colorbar and remove outline
cb = plt.colorbar(label="TWT[s]")
cb.outline.set_visible(False)

# set axes labels and make axes equal
ax.set_xlabel("East")
ax.set_ylabel("North")
ax.axis("equal");

Let's choose a more striking colormap:

In [ ]:
# make figure
fig, ax = plt.subplots(figsize=(10,10))

# draw filled contours
plt.contourf(X, Y, TWT, cmap="turbo")

# draw colorbar and remove outline
cb = plt.colorbar(label="TWT[s]")
cb.outline.set_visible(False)

# set axes labels and make axes equal
ax.set_xlabel("East")
ax.set_ylabel("North")
ax.axis("equal");

This does not look very nice because the shallow areas (low TWT, diapir crests) show blue, while the deep areas (high TWT, minibasins) show red. Let's invert the color map:

In [ ]:
# make figure
fig, ax = plt.subplots(figsize=(10,10))

# draw filled contours with inverted colormap
plt.contourf(X, Y, TWT, cmap="turbo_r") # _r inverts the color map

# draw colorbar and remove outline
cb = plt.colorbar(label="TWT[s]")
cb.outline.set_visible(False)

# set axes labels and make axes equal
ax.set_xlabel("East")
ax.set_ylabel("North")
ax.axis("equal");

That is better, now the deeper areas display blue, while the shallower areas display red. However, the contour interval is too high. Let's fix that by specifying the contour interval and contour levels. We also make the colorbar more reasonable by specifying the ticks values:

In [ ]:
# make levels for filled contours
mi, ma = np.floor(np.nanmin(TWT)), np.ceil(np.nanmax(TWT)) # minimum and maximum TWT values of the surface in s
c_int = 0.1 # contour interval in s
c_levels = np.arange(c_int*(mi//c_int), ma+c_int, c_int) # contour levels, add c_int to ma to include it

# make figure
fig, ax = plt.subplots(figsize=(10,10))

# draw filled contours and specify contour levels
plt.contourf(X, Y, TWT, cmap="turbo_r", levels=c_levels)

# draw colorbar and remove outline, also set tick values
cb = plt.colorbar(label="TWT[s]", ticks=np.arange(0,8,1))
cb.outline.set_visible(False)

# set axes labels and make axes equal
ax.set_xlabel("East")
ax.set_ylabel("North")
ax.axis("equal");

That looks much better 🙂 Now, let's add some contour lines, though we don't want to add too many since the map can get too crowded:

In [ ]:
# make levels for contour lines
c_int = 1 # contour interval in s
cl_levels = np.arange(c_int*(mi//c_int), ma+c_int, c_int) # levels for contour lines, add c_int to ma to include it

# make figure
fig, ax = plt.subplots(figsize=(10,10))

# draw filled contours
plt.contourf(X, Y, TWT, cmap="turbo_r", levels=c_levels)

# draw colorbar and remove outline, also set tick values
cb = plt.colorbar(label="TWT[s]", ticks=np.arange(mi,8,1))
cb.outline.set_visible(False)

# draw contour lines and specify contour levels
ax.contour(X,Y,TWT,levels=cl_levels, linewidths=0.25, linestyles="solid", colors=["black"])

# set axes labels and make axes equal
ax.set_xlabel("East")
ax.set_ylabel("North")
ax.axis("equal"); 

So, the hard part is getting the grid data. Once you have these data, contouring it and making nice figures is easy. In the next section, we will look at how to make grid data from sparse data.

## Interpolating sparse data to create a grid

I often run into the problem of making a grid from an old topographic map, from which there is no published DEM data. The solution to this problem involves digitizing the contour lines of the map, and then interpolating these points to make a grid. Digitizing the contour lines is rather boring, though these days there are wonderful online tools to do this ([this is one](https://automeris.io/WebPlotDigitizer/)). The second part of interpolating the points into a grid, goes in a fly.

Let's look at the following example. This is a topographic map from Ragan's structural geology book. We are going to use this map latter in one exercise:

<img src="../figures/topomap.png" alt="topomap" width="700"/><br><br>

I'll save you the task of digitizing the contour lines. These data are included in the data folder, let's load it:

In [ ]:
ragan_topo = np.loadtxt(os.path.join("..", "data", "ragan_topo.txt"), delimiter=",")
print("size of ragan_topo =", ragan_topo.shape)
ragan_topo

The data consists of 6812 points (6812 rows) with east, north, and up coordinates (3 columns). Let's quickly visualize these data:

In [ ]:
# extract x, y and up coordinates
x_topo = ragan_topo[:,0] # 1st column of ragan_topo
y_topo = ragan_topo[:,1] # 2nd column of ragan_topo
z_topo = ragan_topo[:,2] # 3rd column of ragan_topo

# make figure
fig, ax = plt.subplots(figsize=(14,8))

# plot points
plt.scatter(x_topo, y_topo, c=z_topo, s=1, cmap="gist_earth")

# draw colorbar and remove outline, also set tick values
cb = plt.colorbar(label="Elevation [m]")
cb.outline.set_visible(False)

# set axes labels and make axes equal
ax.set_xlabel("East")
ax.set_ylabel("North")
ax.axis("equal"); 
ax.set_xlim([np.amin(x_topo), np.amax(x_topo)])
ax.set_ylim([np.amin(y_topo), np.amax(y_topo)]);

So, these data are not in a regular grid, but rather the points follow the contour lines. To make them into a regular grid, we must first make a grid, which represents the points we'd like to predict:

In [ ]:
x_points = np.linspace(np.amin(x_topo), np.amax(x_topo), 500) # 500 points along x
y_points = np.linspace(np.amin(y_topo), np.amax(y_topo), 400) # 400 points along y
XG, YG = np.meshgrid(x_points, y_points) # X and Y coordinates of points in 400 x 500 grid
print("size of grid =", XG.shape)

# make figure
fig, ax = plt.subplots(figsize=(14,8))

# plot points of the grid
plt.scatter(XG, YG, s=0.5, color="black")

# set axes labels and make axes equal
ax.set_xlabel("East")
ax.set_ylabel("North")
ax.axis("equal");

Wow, that is a very dense grid of 200,000 points. We need an interpolator to find out the elevation at these points, from the elevation of the points on the contours. Luckily, the `scipy.interpolate.griddata` function does exactly that:

In [ ]:
# import scipy.interpolate
import scipy.interpolate as si

# Interpolate data to make grid, we try the nearest method
# but you can try other such as the linear or cubic methods
ZG = si.griddata((x_topo, y_topo), z_topo, (XG, YG), method="nearest")

# size of ZG
print("size of ZG =", ZG.shape)

Wow, that was fast. Now `XG` (east), `YG` (north), and `ZG` (up) have the same size and represent the coordinates of all grid points. We can now contour these data:

In [ ]:
# contour levels
c_int = 5 # contour interval is 5 m
c_levels = np.arange(c_int*(np.amin(z_topo)//c_int), np.amax(z_topo)+c_int, c_int) # contour levels

# make figure
fig, ax = plt.subplots(figsize=(14,8))

# draw filled contours
plt.contourf(XG, YG, ZG, cmap="gist_earth", levels=c_levels)
plt.colorbar(label="Elevation [m]")

# draw contour lines every 5 m
ax.contour(XG,YG,ZG,levels=c_levels, linewidths=0.5, linestyles="solid", colors=["black"])

# draw thicker contour lines every 25 m and label them
c_int = 25 # contour interval is 25 m
c_levels = np.arange(c_int*(np.amin(z_topo)//c_int), np.amax(z_topo)+c_int, c_int) # contour levels
cs = ax.contour(XG,YG,ZG,levels=c_levels, linewidths=1.0, linestyles="solid", colors=["black"])
ax.clabel(cs)

# set axes labels and make axes equal
ax.set_xlabel("East")
ax.set_ylabel("North")
ax.axis("equal");
ax.set_xlim([np.amin(x_topo), np.amax(x_topo)])
ax.set_ylim([np.amin(y_topo), np.amax(y_topo)]);

Voila... 🙂

Making grids from sparse data is not only useful for extracting DEMs from old topo maps, but for any situation where we have sparse data (or measurements) and we want to interpolate and contour these data.

To practice, try exercise 2 in `day4/lab/lab4.pdf`